In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../results/evaluation_results.csv", sep=";", decimal=",")

In [3]:
def export_latex_table(df, phase, algorithm, filename):
    df_phase = df[(df["phase"] == phase) & (df["algorithm"] == algorithm)].copy()

    df_phase["learning_rate"] = df_phase["learning_rate"].apply(lambda x: f"{x:.0e}")

    # Crear columna de Portfolio Value como mean ± std
    df_phase["PV_raw"] = df_phase["mean_pvs"]  # Guardamos para comparación
    df_phase["PV"] = df_phase.apply(
        lambda row: f"{row['mean_pvs']:.2f} ± {row['std_pvs']:.2f}", axis=1
    )

    # Encontrar los índices de máximos/mínimos
    max_flr_idx = df_phase["final_rewards"].idxmax()
    max_pv_idx = df_phase["PV_raw"].idxmax()
    max_sr_id = df_phase["mean_sharpes"].idxmax()
    min_mdd_idx = df_phase["final_drawdowns"].idxmin()

    # Aplicar negrita a los valores seleccionados
    df_phase["final_rewards"] = df_phase["final_rewards"].apply(
        lambda x: f"\\textbf{{{x:.6f}}}"
        if x == df_phase.loc[max_flr_idx, "final_rewards"]
        else f"{x:.6f}"
    )
    df_phase["PV"] = df_phase.apply(
        lambda row: f"\\textbf{{{row['PV']}}}" if row.name == max_pv_idx else row["PV"],
        axis=1,
    )
    df_phase["mean_sharpes"] = df_phase["mean_sharpes"].apply(
        lambda x: f"\\textbf{{{x:.6f}}}"
        if x == df_phase.loc[max_sr_id, "mean_sharpes"]
        else f"{x:.6f}"
    )
    df_phase["final_drawdowns"] = df_phase["final_drawdowns"].apply(
        lambda x: f"\\textbf{{{x:.6f}}}"
        if x == df_phase.loc[min_mdd_idx, "final_drawdowns"]
        else f"{x:.6f}"
    )

    # Seleccionar y renombrar columnas
    df_out = df_phase[
        [
            "algorithm",
            "feature",
            "number_of_companies",
            "learning_rate",
            "mean_sharpes",
            "final_drawdowns",
            "PV",
            "final_rewards",
        ]
    ].rename(
        columns={
            "algorithm": "Algoritmo",
            "feature": "FE",
            "number_of_companies": "N",
            "learning_rate": "lr",
            "mean_sharpes": "SR",
            "final_drawdowns": "MDD",
            "final_rewards": "fLR",
            "PV": "PV",
        }
    )

    df_out.sort_values(
        by=["Algoritmo", "FE", "N", "lr"],
        ascending=[False, False, False, False],
        inplace=True,
    )

    # Exportar a LaTeX
    phase_map = {
        "train": "Entrenamiento",
        "eval": "Test",
    }
    df_out.to_latex(
        filename,
        index=False,
        escape=False,
        caption=f"Resultados para la Fase de  {phase_map[phase]} con el algoritmo {algorithm}",
        label=f"tab:{phase}_results_{algorithm}",
    )

In [5]:
# Usar la función para ambos phases
export_latex_table(
    df,
    phase="train",
    algorithm="DDPG",
    filename="../results/Latex/train_ddpg.tex",
)
export_latex_table(
    df,
    phase="train",
    algorithm="TD3",
    filename="../results/Latex/train_td3.tex",
)
export_latex_table(
    df,
    phase="eval",
    algorithm="DDPG",
    filename="../results/Latex/test_ddpg.tex",
)
export_latex_table(
    df,
    phase="eval",
    algorithm="TD3",
    filename="../results/Latex/test_td3.tex",
)
